## <center> Модели CatBoost

Так как бустинг - довольно долгая процедура, а последовательность PPH-PPH-PPH-PPH мы предсказываем довольно неплохо, то исключим ее из построения. Исключаем после деления на train/test, чтобы деление оставалось такое же, как и в предыдущих моделях для сравнимости результатов.

Были подобраны параметры обучения, которые поднимают качество, и при этом модель считается за приемлемое (10-15 минут) время. 

При увеличении параметра depth качество растет, но и время тоже. Также рост будет при увеличении iterations. Но тогда время будет совсем большим. На видеокарте, как отмечено в комментариях кода, расчет будет гораздо быстрее, но каждый раз с новым результатом.

Построим модели по департаметам:

Загрузка библиотек

In [1]:
import numpy as np #массивы
import pandas as pd #работа с данными
import pickle #загрузка-выгрузка pickle-файлов

from sklearn import metrics #метрики
from sklearn.model_selection import train_test_split #деление выборки на train/test

from catboost import CatBoostClassifier #бустинг

from IPython.display import display, HTML #параметры отображения

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 1000)

Подготовка базы для моделирования

In [2]:
%%time

my_random = 42 #случайное число, если процедура позволяет или требует его задать

#загрузка базы для модели
with open('./files/df_all_model.pkl', 'rb') as pkl_file:
    df_all_model = pickle.load(pkl_file)

#загрузка базы с разметкой по модели случайного леса, кодом департамента и последовательностью растений
with open('./files/y_forest_dep.pkl', 'rb') as pkl_file:
    y_forest_dep = pickle.load(pkl_file)

#список переменных для моделирования
var_list = [
    'gr t-1_0', 'gr t-1_1', 'gr t-1_2', 'gr t-1_3', 'gr t-1_4',
    'gr t-2_0', 'gr t-2_1', 'gr t-2_2', 'gr t-2_3', 'gr t-2_4',
    'gr t-3_0', 'gr t-3_1', 'gr t-3_2', 'gr t-3_3', 'gr t-3_4',
    'gr t-4_0', 'gr t-4_1', 'gr t-4_2', 'gr t-4_3', 'gr t-4_4',
    'cult t-1_0', 'cult t-1_1', 'cult t-1_2', 'cult t-1_3', 'cult t-1_4',
    'cult t-1_5', 'cult t-1_6', 'cult t-1_7', 'cult t-1_8',
    'cult t-2_0', 'cult t-2_1', 'cult t-2_2', 'cult t-2_3', 'cult t-2_4',
    'cult t-2_5', 'cult t-2_6', 'cult t-2_7', 'cult t-2_8',
    'cult t-3_0', 'cult t-3_1', 'cult t-3_2', 'cult t-3_3', 'cult t-3_4',
    'cult t-3_5', 'cult t-3_6', 'cult t-3_7',
    'cult t-4_0', 'cult t-4_1', 'cult t-4_2', 'cult t-4_3', 'cult t-4_4', 
    'cult t-4_5', 'cult t-4_6', 'cult t-4_7',
    'DEP_RATTAC_0', 'DEP_RATTAC_1', 'DEP_RATTAC_2', 'DEP_RATTAC_3', 'DEP_RATTAC_4', 'DEP_RATTAC_5', 'DEP_RATTAC_6',
    'x_round', 'y_round'
]

target = 'CODE_CULTU'

#выборка для построения
X = df_all_model[df_all_model['type'] == 'modelling'][var_list + ['culture_str'] + ['DEP_RATTAC']]

#целевая переменная
y = df_all_model[df_all_model['type'] == 'modelling'][target]

#train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=my_random, test_size=0.2)

#список департаментов
department_list = df_all_model['DEP_RATTAC'].unique()
department_list.sort()
print(department_list)

['001' '002' '003' '004' '005' '006' '007' '008' '009' '010' '011' '012'
 '013' '014' '015' '016' '017' '018' '019' '021' '022' '023' '024' '025'
 '026' '027' '028' '029' '02A' '02B' '030' '031' '032' '033' '034' '035'
 '036' '037' '038' '039' '040' '041' '042' '043' '044' '045' '046' '047'
 '048' '049' '050' '051' '052' '053' '054' '055' '056' '057' '058' '059'
 '060' '061' '062' '063' '064' '065' '066' '067' '068' '069' '070' '071'
 '072' '073' '074' '076' '077' '078' '079' '080' '081' '082' '083' '084'
 '085' '086' '087' '088' '089' '090' '091' '094' '095']
CPU times: total: 7.16 s
Wall time: 7.15 s


In [3]:
%%time

#обработка баз train и test
#флаг train/test
X_train['train'] = 1
X_test['train'] = 0

#флаг наличия последовательности культур 'PPH-PPH-PPH-PPH'
X_train['PPH'] = X_train['culture_str'].apply(lambda x: 1 if x == 'PPH-PPH-PPH-PPH' else 0)
X_test['PPH'] = X_test['culture_str'].apply(lambda x: 1 if x == 'PPH-PPH-PPH-PPH' else 0)

X_y_train = pd.concat([X_train, y_train], axis = 1)
X_y_test = pd.concat([X_test, y_test], axis = 1)

X_y = pd.concat([X_y_train, X_y_test], axis = 0)

#переменные для удаления из X
del_vars = ['CODE_CULTU', 'DEP_RATTAC', 'culture_str', 'train', 'PPH']

CPU times: total: 1.64 s
Wall time: 1.63 s


In [4]:
%%time

#функция построения модели и расчета результатов по департаменту с помощью CatBoost
def department_boosting(department):
    #отбор департамента
    X_y_dep = X_y[X_y['DEP_RATTAC'] == department].copy()
    
    print(f"департамент {department}. наблюдений: {X_y_dep[(X_y_dep['train'] == 1) & (X_y_dep['PPH'] == 0)].shape[0]} ")

    #задание параметров модели
    boost_model = CatBoostClassifier(
        #task_type='GPU', - быстрее в 10+ раз, но результаты невоспроизводимы:
        #https://catboost.ai/en/docs/features/training-on-gpu - Training on GPU is non-deterministic
        loss_function='MultiClass',
        random_seed=my_random,
        max_depth=9,
        thread_count=12,
        iterations=250,
        verbose=0
    )

    #построение модели
    boost_model.fit(
        X_y_dep[(X_y_dep['train'] == 1) & (X_y_dep['PPH'] == 0)].drop(columns=del_vars),
        X_y_dep[(X_y_dep['train'] == 1) & (X_y_dep['PPH'] == 0)]['CODE_CULTU']
    )

    #применение модели, добавление столбца с предсказанием
    y_pred = boost_model.predict(X_y_dep.drop(columns=del_vars))
    X_y_dep['predict_boost'] = y_pred.copy()

    #пересчет предсказания для строк 'PPH-PPH-PPH-PPH'
    def pph_add(x):
        if x['culture_str'] == 'PPH-PPH-PPH-PPH':
            predict_final = 'PPH'
        else:
            predict_final = x['predict_boost']
        return predict_final

    X_y_dep['predict_final'] = X_y_dep.apply(lambda x: pph_add(x), axis=1)

    #выделение X и y в train/test
    y_train_calc = X_y_dep[X_y_dep['train'] == 1]['predict_final']
    y_train_fact = X_y_dep[X_y_dep['train'] == 1]['CODE_CULTU']
    y_test_calc = X_y_dep[X_y_dep['train'] == 0]['predict_final']
    y_test_fact = X_y_dep[X_y_dep['train'] == 0]['CODE_CULTU']

    print(f"accuracy на train: {metrics.accuracy_score(y_train_calc, y_train_fact) * 100:.2f}%")
    print(f"accuracy на test: {metrics.accuracy_score(y_test_calc, y_test_fact) * 100:.2f}%")
    
    #выгрузка модели
    with open(f"./boosting/boost_dep_{department}.pkl", 'wb') as output:
        str_ = f"pickle.dump(boost_model, output)"
        exec(str_)
    
    #выгрузка результатов
    result = X_y_dep[['train', 'PPH', 'CODE_CULTU', 'predict_boost', 'predict_final']]
    with open(f"./boosting/calc_dep_{department}.pkl", 'wb') as output:
        pickle.dump(result, output)
        
    print('-----' * 10)

CPU times: total: 0 ns
Wall time: 0 ns


In [5]:
%%time

#список департаментов
department_list = X['DEP_RATTAC'].unique()
department_list.sort()
print(f"всего департаментов {len(department_list)}")
print(department_list)

всего департаментов 93
['001' '002' '003' '004' '005' '006' '007' '008' '009' '010' '011' '012'
 '013' '014' '015' '016' '017' '018' '019' '021' '022' '023' '024' '025'
 '026' '027' '028' '029' '02A' '02B' '030' '031' '032' '033' '034' '035'
 '036' '037' '038' '039' '040' '041' '042' '043' '044' '045' '046' '047'
 '048' '049' '050' '051' '052' '053' '054' '055' '056' '057' '058' '059'
 '060' '061' '062' '063' '064' '065' '066' '067' '068' '069' '070' '071'
 '072' '073' '074' '076' '077' '078' '079' '080' '081' '082' '083' '084'
 '085' '086' '087' '088' '089' '090' '091' '094' '095']
CPU times: total: 93.8 ms
Wall time: 94 ms


Будем запускать порциями по 5 департаментов в одной ячейке - ~60 минут на порцию, 19 порций. Больше для удобства, чем для какой-то практической цели.

In [6]:
%%time
part = 1
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 001. наблюдений: 20109 
accuracy на train: 92.63%
accuracy на test: 79.90%
--------------------------------------------------
департамент 002. наблюдений: 21026 
accuracy на train: 92.18%
accuracy на test: 78.99%
--------------------------------------------------
департамент 003. наблюдений: 12881 
accuracy на train: 96.66%
accuracy на test: 83.81%
--------------------------------------------------
департамент 004. наблюдений: 19395 
accuracy на train: 95.76%
accuracy на test: 84.46%
--------------------------------------------------
департамент 005. наблюдений: 10800 
accuracy на train: 97.53%
accuracy на test: 88.83%
--------------------------------------------------
CPU times: total: 2h 28min 27s
Wall time: 24min 43s


In [7]:
%%time
part = 2
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 006. наблюдений: 244 
accuracy на train: 99.34%
accuracy на test: 94.59%
--------------------------------------------------
департамент 007. наблюдений: 21716 
accuracy на train: 97.47%
accuracy на test: 89.21%
--------------------------------------------------
департамент 008. наблюдений: 10518 
accuracy на train: 96.88%
accuracy на test: 82.89%
--------------------------------------------------
департамент 009. наблюдений: 9483 
accuracy на train: 98.95%
accuracy на test: 89.28%
--------------------------------------------------
департамент 010. наблюдений: 22250 
accuracy на train: 90.73%
accuracy на test: 75.08%
--------------------------------------------------
CPU times: total: 2h 2min 40s
Wall time: 21min 29s


In [8]:
%%time
part = 3
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 011. наблюдений: 34591 
accuracy на train: 96.09%
accuracy на test: 89.20%
--------------------------------------------------
департамент 012. наблюдений: 57027 
accuracy на train: 89.63%
accuracy на test: 81.89%
--------------------------------------------------
департамент 013. наблюдений: 755 
accuracy на train: 100.00%
accuracy на test: 79.43%
--------------------------------------------------
департамент 014. наблюдений: 15496 
accuracy на train: 93.44%
accuracy на test: 83.34%
--------------------------------------------------
департамент 015. наблюдений: 13660 
accuracy на train: 97.99%
accuracy на test: 94.85%
--------------------------------------------------
CPU times: total: 2h 38min 42s
Wall time: 43min 41s


In [9]:
%%time
part = 4
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 016. наблюдений: 47857 
accuracy на train: 90.79%
accuracy на test: 81.69%
--------------------------------------------------
департамент 017. наблюдений: 53699 
accuracy на train: 87.84%
accuracy на test: 78.07%
--------------------------------------------------
департамент 018. наблюдений: 21879 
accuracy на train: 94.17%
accuracy на test: 80.58%
--------------------------------------------------
департамент 019. наблюдений: 15767 
accuracy на train: 97.92%
accuracy на test: 92.54%
--------------------------------------------------
департамент 021. наблюдений: 25599 
accuracy на train: 92.54%
accuracy на test: 82.30%
--------------------------------------------------
CPU times: total: 3h 59min 13s
Wall time: 1h 24min 23s


In [10]:
%%time
part = 5
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 022. наблюдений: 56547 
accuracy на train: 80.58%
accuracy на test: 71.44%
--------------------------------------------------
департамент 023. наблюдений: 27400 
accuracy на train: 94.30%
accuracy на test: 88.44%
--------------------------------------------------
департамент 024. наблюдений: 34764 
accuracy на train: 95.06%
accuracy на test: 84.37%
--------------------------------------------------
департамент 025. наблюдений: 6720 
accuracy на train: 98.56%
accuracy на test: 92.32%
--------------------------------------------------
департамент 026. наблюдений: 28758 
accuracy на train: 95.56%
accuracy на test: 83.14%
--------------------------------------------------
CPU times: total: 3h 54min 24s
Wall time: 1h 24min 48s


In [11]:
%%time
part = 6
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 027. наблюдений: 18822 
accuracy на train: 90.20%
accuracy на test: 79.65%
--------------------------------------------------
департамент 028. наблюдений: 20516 
accuracy на train: 89.79%
accuracy на test: 77.84%
--------------------------------------------------
департамент 029. наблюдений: 36090 
accuracy на train: 85.78%
accuracy на test: 70.70%
--------------------------------------------------
департамент 02A. наблюдений: 447 
accuracy на train: 99.83%
accuracy на test: 94.25%
--------------------------------------------------
департамент 02B. наблюдений: 666 
accuracy на train: 99.61%
accuracy на test: 95.28%
--------------------------------------------------
CPU times: total: 2h 11min 30s
Wall time: 39min 14s


In [12]:
%%time
part = 7
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 030. наблюдений: 32116 
accuracy на train: 96.61%
accuracy на test: 87.31%
--------------------------------------------------
департамент 031. наблюдений: 29313 
accuracy на train: 92.21%
accuracy на test: 79.63%
--------------------------------------------------
департамент 032. наблюдений: 43314 
accuracy на train: 89.93%
accuracy на test: 76.70%
--------------------------------------------------
департамент 033. наблюдений: 20820 
accuracy на train: 98.29%
accuracy на test: 92.43%
--------------------------------------------------
департамент 034. наблюдений: 39422 
accuracy на train: 95.85%
accuracy на test: 91.53%
--------------------------------------------------
CPU times: total: 3h 57min 39s
Wall time: 1h 29min 35s


In [13]:
%%time
part = 8
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 035. наблюдений: 33242 
accuracy на train: 88.11%
accuracy на test: 75.80%
--------------------------------------------------
департамент 036. наблюдений: 29688 
accuracy на train: 93.18%
accuracy на test: 80.56%
--------------------------------------------------
департамент 037. наблюдений: 26907 
accuracy на train: 92.75%
accuracy на test: 77.17%
--------------------------------------------------
департамент 038. наблюдений: 32266 
accuracy на train: 92.36%
accuracy на test: 83.40%
--------------------------------------------------
департамент 039. наблюдений: 11112 
accuracy на train: 97.81%
accuracy на test: 88.43%
--------------------------------------------------
CPU times: total: 3h 23min 52s
Wall time: 1h 5min


In [14]:
%%time
part = 9
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 040. наблюдений: 21957 
accuracy на train: 94.48%
accuracy на test: 85.62%
--------------------------------------------------
департамент 041. наблюдений: 15240 
accuracy на train: 94.95%
accuracy на test: 77.52%
--------------------------------------------------
департамент 042. наблюдений: 19521 
accuracy на train: 94.99%
accuracy на test: 84.02%
--------------------------------------------------
департамент 043. наблюдений: 30031 
accuracy на train: 94.63%
accuracy на test: 83.96%
--------------------------------------------------
департамент 044. наблюдений: 30507 
accuracy на train: 89.30%
accuracy на test: 75.24%
--------------------------------------------------
CPU times: total: 3h 20min 5s
Wall time: 56min 41s


In [15]:
%%time
part = 10
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 045. наблюдений: 21906 
accuracy на train: 92.41%
accuracy на test: 74.89%
--------------------------------------------------
департамент 046. наблюдений: 36022 
accuracy на train: 94.28%
accuracy на test: 87.34%
--------------------------------------------------
департамент 047. наблюдений: 26100 
accuracy на train: 93.56%
accuracy на test: 79.31%
--------------------------------------------------
департамент 048. наблюдений: 32258 
accuracy на train: 96.40%
accuracy на test: 91.03%
--------------------------------------------------
департамент 049. наблюдений: 29062 
accuracy на train: 93.12%
accuracy на test: 75.84%
--------------------------------------------------
CPU times: total: 3h 30min 16s
Wall time: 1h 7min 23s


In [16]:
%%time
part = 11
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 050. наблюдений: 33537 
accuracy на train: 88.64%
accuracy на test: 82.01%
--------------------------------------------------
департамент 051. наблюдений: 33916 
accuracy на train: 88.59%
accuracy на test: 75.20%
--------------------------------------------------
департамент 052. наблюдений: 9735 
accuracy на train: 96.75%
accuracy на test: 83.49%
--------------------------------------------------
департамент 053. наблюдений: 9382 
accuracy на train: 94.67%
accuracy на test: 79.31%
--------------------------------------------------
департамент 054. наблюдений: 13367 
accuracy на train: 95.48%
accuracy на test: 83.77%
--------------------------------------------------
CPU times: total: 2h 28min 53s
Wall time: 39min 39s


In [17]:
%%time
part = 12
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 055. наблюдений: 8062 
accuracy на train: 97.96%
accuracy на test: 83.14%
--------------------------------------------------
департамент 056. наблюдений: 23329 
accuracy на train: 89.61%
accuracy на test: 73.04%
--------------------------------------------------
департамент 057. наблюдений: 15724 
accuracy на train: 96.32%
accuracy на test: 85.67%
--------------------------------------------------
департамент 058. наблюдений: 12395 
accuracy на train: 96.95%
accuracy на test: 86.81%
--------------------------------------------------
департамент 059. наблюдений: 23774 
accuracy на train: 90.45%
accuracy на test: 76.52%
--------------------------------------------------
CPU times: total: 2h 27min
Wall time: 32min 56s


In [18]:
%%time
part = 13
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 060. наблюдений: 19743 
accuracy на train: 88.16%
accuracy на test: 76.31%
--------------------------------------------------
департамент 061. наблюдений: 17471 
accuracy на train: 93.89%
accuracy на test: 84.35%
--------------------------------------------------
департамент 062. наблюдений: 31877 
accuracy на train: 88.60%
accuracy на test: 75.95%
--------------------------------------------------
департамент 063. наблюдений: 27041 
accuracy на train: 94.53%
accuracy на test: 88.70%
--------------------------------------------------
департамент 064. наблюдений: 51949 
accuracy на train: 92.88%
accuracy на test: 89.06%
--------------------------------------------------
CPU times: total: 3h 8min 35s
Wall time: 1h 1min 1s


In [19]:
%%time
part = 14
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 065. наблюдений: 18733 
accuracy на train: 95.34%
accuracy на test: 87.37%
--------------------------------------------------
департамент 066. наблюдений: 16277 
accuracy на train: 98.39%
accuracy на test: 94.82%
--------------------------------------------------
департамент 067. наблюдений: 44922 
accuracy на train: 86.55%
accuracy на test: 81.98%
--------------------------------------------------
департамент 068. наблюдений: 29245 
accuracy на train: 92.12%
accuracy на test: 85.00%
--------------------------------------------------
департамент 069. наблюдений: 15308 
accuracy на train: 96.84%
accuracy на test: 84.01%
--------------------------------------------------
CPU times: total: 3h 1min 8s
Wall time: 53min 19s


In [20]:
%%time
part = 15
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 070. наблюдений: 11185 
accuracy на train: 96.90%
accuracy на test: 84.71%
--------------------------------------------------
департамент 071. наблюдений: 23260 
accuracy на train: 96.14%
accuracy на test: 87.99%
--------------------------------------------------
департамент 072. наблюдений: 16591 
accuracy на train: 93.44%
accuracy на test: 79.80%
--------------------------------------------------
департамент 073. наблюдений: 6745 
accuracy на train: 98.66%
accuracy на test: 94.45%
--------------------------------------------------
департамент 074. наблюдений: 8697 
accuracy на train: 97.68%
accuracy на test: 91.91%
--------------------------------------------------
CPU times: total: 2h 1min 50s
Wall time: 22min 20s


In [21]:
%%time
part = 16
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 076. наблюдений: 13061 
accuracy на train: 94.10%
accuracy на test: 79.86%
--------------------------------------------------
департамент 077. наблюдений: 20072 
accuracy на train: 91.33%
accuracy на test: 75.43%
--------------------------------------------------
департамент 078. наблюдений: 6905 
accuracy на train: 96.05%
accuracy на test: 77.30%
--------------------------------------------------
департамент 079. наблюдений: 36285 
accuracy на train: 89.54%
accuracy на test: 73.39%
--------------------------------------------------
департамент 080. наблюдений: 25423 
accuracy на train: 86.90%
accuracy на test: 71.50%
--------------------------------------------------
CPU times: total: 2h 54min 55s
Wall time: 47min 36s


In [22]:
%%time
part = 17
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 081. наблюдений: 30412 
accuracy на train: 93.52%
accuracy на test: 77.46%
--------------------------------------------------
департамент 082. наблюдений: 25184 
accuracy на train: 93.83%
accuracy на test: 81.31%
--------------------------------------------------
департамент 083. наблюдений: 849 
accuracy на train: 99.89%
accuracy на test: 85.45%
--------------------------------------------------
департамент 084. наблюдений: 1760 
accuracy на train: 99.18%
accuracy на test: 83.17%
--------------------------------------------------
департамент 085. наблюдений: 26318 
accuracy на train: 92.25%
accuracy на test: 77.44%
--------------------------------------------------
CPU times: total: 2h 53min 50s
Wall time: 53min 43s


In [23]:
%%time
part = 18
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 086. наблюдений: 36666 
accuracy на train: 88.51%
accuracy на test: 72.60%
--------------------------------------------------
департамент 087. наблюдений: 16176 
accuracy на train: 93.66%
accuracy на test: 82.08%
--------------------------------------------------
департамент 088. наблюдений: 8715 
accuracy на train: 97.71%
accuracy на test: 89.12%
--------------------------------------------------
департамент 089. наблюдений: 30815 
accuracy на train: 88.54%
accuracy на test: 75.11%
--------------------------------------------------
департамент 090. наблюдений: 1486 
accuracy на train: 99.73%
accuracy на test: 86.36%
--------------------------------------------------
CPU times: total: 2h 39min 6s
Wall time: 48min 51s


In [24]:
%%time
part = 19
for department in department_list[(part - 1) * 5:min(part * 5, len(department_list))]:
    department_boosting(department=department)

департамент 091. наблюдений: 6571 
accuracy на train: 96.59%
accuracy на test: 75.77%
--------------------------------------------------
департамент 094. наблюдений: 104 
accuracy на train: 100.00%
accuracy на test: 76.47%
--------------------------------------------------
департамент 095. наблюдений: 2078 
accuracy на train: 99.35%
accuracy на test: 77.35%
--------------------------------------------------
CPU times: total: 39min 4s
Wall time: 5min 39s
